In [6]:
# попробовала доп. задание 7. Только на бустинге
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [8]:
# Загрузка данных
data = pd.read_csv('telcom_customer_churn.csv')

numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
data[numeric_cols] = data[numeric_cols].fillna(data[numeric_cols].mean())

# Заполнение NaN в строковых столбцах модой
for column in data.select_dtypes(include=['object']).columns:
    mode_value = data[column].mode()[0]  # Вычисляем моду
    data[column] = data[column].fillna(mode_value)  # Заполняем NaN модой

In [12]:
# Определяем целевую переменную и признаки
X = data.drop('churn', axis=1)
y = data['churn']

# Создание новых признаков
new_features = pd.DataFrame({
    'mou_ratio': X['mou_Mean'] / (X['change_mou'] + 1e-5),  # Добавляем небольшое значение для избежания деления на ноль
    'avg_usage': X['avgmou'] * X['avgqty'],
    'months_squared': X['months'] ** 2
})


In [16]:
# Объединяем новые признаки с оригинальным DataFrame
X = pd.concat([X, new_features], axis=1)

# Разделение данных на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Определение числовых и категориальных признаков
num_features = X.select_dtypes(include=['int64', 'float64']).columns
cat_features = X.select_dtypes(include=['object']).columns


In [18]:
# Сравнение методов кодирования
# One-Hot Encoding
preprocessor_onehot = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
    ])

# Label Encoding (для примера, только для одной колонки)
label_encoded_X = X.copy()
label_encoder = LabelEncoder()
for col in cat_features:
    label_encoded_X[col] = label_encoder.fit_transform(label_encoded_X[col])

In [20]:
# Разделение данных для Label Encoding
X_train_label, X_test_label, y_train_label, y_test_label = train_test_split(label_encoded_X, y, test_size=0.3, random_state=42)

# Обучение модели с One-Hot Encoding
gb_pipeline_onehot = Pipeline(steps=[
    ('preprocessor', preprocessor_onehot),
    ('classifier', GradientBoostingClassifier(random_state=42))
])

gb_pipeline_onehot.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['rev_Mean', 'mou_Mean', 'totmrc_Mean', 'da_Mean', 'ovrmou_Mean',
       'ovrrev_Mean', 'vceovr_Mean', 'datovr_Mean', 'roam_Mean', 'change_mou',
       'change_rev', 'drop_vce_Mean', 'drop_dat_Mean', 'blck_vce_Mean',
       'blck_dat_Mean', 'unan_vce_Mean', 'unan_dat_Mean', 'plcd_vce_Mean',
       'pl...
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['new_cell', 'crclscod', 'asl_flag', 'prizm_social_one', 'area',
       'dualband', 'refurb_new', 'hnd_webcap', 'ownrent', 'dwlltype',
       'marital', 'infobase', 'HHstatin', 'dwllsize', 'ethnic', 'kid0_2',
       'kid3_5', 'kid6_10', 'kid11_15', 'kid16_17', 'creditcd'],
      dtype='object'))])),
                ('classifier', GradientBoostingClassifier(random_state=42))])

In [22]:
# Оценка производительности с One-Hot Encoding
y_pred_onehot = gb_pipeline_onehot.predict(X_test)
metrics_onehot = {
    'Accuracy': accuracy_score(y_test, y_pred_onehot),
    'Precision': precision_score(y_test, y_pred_onehot),
    'Recall': recall_score(y_test, y_pred_onehot),
    'F1 Score': f1_score(y_test, y_pred_onehot),
    'ROC AUC': roc_auc_score(y_test, gb_pipeline_onehot.predict_proba(X_test)[:, 1])
}

In [24]:
# Обучение модели с Label Encoding
gb_pipeline_label = Pipeline(steps=[
    ('preprocessor', StandardScaler()),  # Только числовые признаки
    ('classifier', GradientBoostingClassifier(random_state=42))
])

gb_pipeline_label.fit(X_train_label, y_train_label)


Pipeline(steps=[('preprocessor', StandardScaler()),
                ('classifier', GradientBoostingClassifier(random_state=42))])

In [26]:
# Оценка производительности с Label Encoding
y_pred_label = gb_pipeline_label.predict(X_test_label)
metrics_label = {
    'Accuracy': accuracy_score(y_test_label, y_pred_label),
    'Precision': precision_score(y_test_label, y_pred_label),
    'Recall': recall_score(y_test_label, y_pred_label),
    'F1 Score': f1_score(y_test_label, y_pred_label),
    'ROC AUC': roc_auc_score(y_test_label, gb_pipeline_label.predict_proba(X_test_label)[:, 1])
}

# Вывод результатов
print("Metrics with One-Hot Encoding:", metrics_onehot)
print("Metrics with Label Encoding:", metrics_label)

Metrics with One-Hot Encoding: {'Accuracy': 0.6271666666666667, 'Precision': 0.6217189912506433, 'Recall': 0.645600908544325, 'F1 Score': 0.6334349293743651, 'ROC AUC': 0.6817450473644022}
Metrics with Label Encoding: {'Accuracy': 0.6292, 'Precision': 0.622992770776022, 'Recall': 0.6505444585476652, 'F1 Score': 0.6364705882352941, 'ROC AUC': 0.6819945239855003}


In [28]:
# Создание DataFrame для сравнения метрик
comparison_df = pd.DataFrame({
    'Metric': metrics_onehot.keys(),
    'One-Hot Encoding': metrics_onehot.values(),
    'Label Encoding': metrics_label.values()
})

# Вывод результатов
print(comparison_df)

      Metric  One-Hot Encoding  Label Encoding
0   Accuracy          0.627167        0.629200
1  Precision          0.621719        0.622993
2     Recall          0.645601        0.650544
3   F1 Score          0.633435        0.636471
4    ROC AUC          0.681745        0.681995


In [ ]:
''' таким образом по всем показателям Label Encoding превосходит One-Hot Encoding

1  Accuracy: Оба метода имеют схожую точность, но Label Encoding немного лучше (0.629 против 0.627).

2. Precision: Разница между метриками минимальная, но Label Encoding также показывает небольшое преимущество.

3. Recall: Label Encoding имеет более высокий показатель, что может указывать на то, что модель лучше выявляет положительные случаи.

4. F1 Score: Снова Label Encoding показывает лучшее значение, что говорит о лучшем балансе между точностью и полнотой.

5. ROC AUC: Оба метода имеют схожие значения, но Label Encoding немного опережает One-Hot Encoding '''
